In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from collections import OrderedDict
import matplotlib as mpl
import matplotlib.patches as mpatches
import smopy
import json
mpl.style.use('seaborn')
end_date = datetime.datetime.now()
last_monday = end_date - datetime.timedelta(days=end_date.weekday())
start_date = last_monday - datetime.timedelta(21)
dt_format = "%Y%m%d%H%M%S"

cameraFriendlyNames = {
    'PER_PEOPLE_BLACKETT-NORTHUMBERLAND-W': 'Blackett St pavement (north side) outside Rox',
    'PER_PEOPLE_BLACKETT-BOOTS': 'Blackett St outside Boots',
    'PER_PEOPLE_THE_CORE_LINE_0': 'Blue Star Square at Newcastle Helix (east side)',
    'PER_PEOPLE_THE_CORE_LINE_1': 'Blue Star Square at Newcastle Helix (west side)',
    'PER_PEOPLE_USB_LINE_0': 'Science Square at Newcastle Helix',
    'PER_PEOPLE_NORTHUMERLAND_LINE_LONG_DISTANCE_HEAD_0': 'Northumberland St near Fenwick (west side)',
    'PER_PEOPLE_NORTHUMERLAND_LINE_LONG_DISTANCE_HEAD_1': 'Northumberland St near Fenwick (east side)',
    'PER_PEOPLE_NORTHUMERLAND_LINE_MID_DISTANCE_HEAD_0': 'Northumberland St near TK Maxx',
    'PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_0': 'John Dobson St (west side) pavement near Goldsmiths',
    'PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_1': 'John Dobson St crossing island between Blackett St and New Bridge St West',
    'PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_2': 'John Dobson St (east side) pavement near The Stack',
    'PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_3': 'Pavement (south side) corner John Dobson St and Blackett St',
    'PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_4': 'Pavement (south side) corner John Dobson St and New Bridge St West',
    'PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_5': 'Blackett St crossing from John Dobson St to Northumberland St (west side)',
    'PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_6': 'New Bridge St West crossing John Dobson St to Northumberland St (east side)'
}

dt_format = "%Y%m%d%H%M%S"


patterns = ['ooo','+++', 'xxx', '\\\\', '...']

for db_name,pretty_name in cameraFriendlyNames.items():
    
    geojson = json.load(open('../cache/bar-charts/people-geojson/{sensor_name}.json'.format(sensor_name=db_name)))

    map = smopy.Map(
        (
            geojson['coordinates'][1]-0.00001,
            geojson['coordinates'][0]-0.00001,
            geojson['coordinates'][1]+0.00001,
            geojson['coordinates'][0]+0.00001
        ),
        z=40)

    fig, ax2 = plt.subplots(figsize=(5, 5),dpi=300)
    x, y = map.to_pixels(geojson['coordinates'][1], geojson['coordinates'][0])
    ax = map.show_mpl(ax2)
    ax.plot(x, y, 'or', ms=5, mew=1);
    plt.title(pretty_name)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.xlim(0, map.w)
    plt.ylim(map.h, 0)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    df = pd.read_pickle('../cache/bar-charts/people-data/{sensor_name}.tar.gz'.format(sensor_name=db_name))
    df.index = pd.to_datetime(df['Timestamp'])
    if df.empty:
        continue
    df['dayofweek']=df.index.weekday
    df['week']= df.index.week
    bars = OrderedDict()
    for week,week_frame in df.groupby('week'):
        day_values = OrderedDict(zip(range(7),[None]*7))
        for dayofweek, day_frame in week_frame.groupby('dayofweek'):

            day_values[dayofweek] = [0]*5
            day_values[dayofweek][0] = day_frame[day_frame.index.time<datetime.time(7,0)]['Value'].sum()
            day_values[dayofweek][1] = day_frame[(day_frame.index.time < datetime.time(10,0)) & (day_frame.index.time >= datetime.time(7,0))]['Value'].sum()
            day_values[dayofweek][2] = day_frame[
                (day_frame.index.time >= datetime.time(10, 0)) & (day_frame.index.time < datetime.time(16, 0))][
                'Value'].sum()
            day_values[dayofweek][3] = day_frame[
                (day_frame.index.time >= datetime.time(16, 0)) & (day_frame.index.time < datetime.time(19, 0))][
                'Value'].sum()
            day_values[dayofweek][4] = day_frame[
                day_frame.index.time >= datetime.time(19, 0)][
                'Value'].sum()
        bars[week] = list(day_values.values())
    # set width of bar
    barWidth = 0.2
    

    fig, ax1 = plt.subplots(figsize=(20, 20),dpi=300)
    for i,(week_num,sub_bars) in enumerate(bars.items()):
        for j,stack in enumerate(sub_bars):
            if stack == None:
                stack = [0,]*5
            for k,bar in enumerate(stack):

                ax1.bar(j+barWidth*i,
                        bar,
                        bottom=sum(stack[:k]),
                        width=barWidth,
                        color='white',
                        edgecolor='C'+str(i),
                        hatch=patterns[k],
                        linewidth=1
                        )



    

    # Add xticks on the middle of the group bars
    ax1.set_ylabel('People', fontweight='bold')
    ax1.set_xlabel('Day of the Week', fontweight='bold')

    ax1.set_xticks([r + (barWidth*1.5) for r in range(7)])
    ax1.set_xticklabels(['Mon','Tue','Wed','Thu','Fri','Sat','Sun'])

    # Create legend & Show graphic
    legend_items = []
    for i in range(4):
        label = 'Week of ' + (last_monday - datetime.timedelta(weeks=3-i)).strftime('%a %e %B')
        legend_items.append(mpatches.Patch(facecolor='white',edgecolor='C'+str(i),linewidth=1,label=label))

    hatch_times = [
        datetime.datetime(1,1,1,0,0),
        datetime.datetime(1, 1, 1, 7, 0),
        datetime.datetime(1, 1, 1, 10, 0),
        datetime.datetime(1, 1, 1, 16, 0),
        datetime.datetime(1, 1, 1, 19, 0),
        datetime.datetime(1, 1, 1, 0, 0),
    ]
    hatch_labels = [
        hatch_times[0].strftime('%R') + ' - ' +hatch_times[1].strftime('%R'),
        hatch_times[1].strftime('%R') + ' - ' + hatch_times[2].strftime('%R'),
        hatch_times[2].strftime('%R') + ' - ' + hatch_times[3].strftime('%R'),
        hatch_times[3].strftime('%R') + ' - ' + hatch_times[4].strftime('%R'),
        hatch_times[4].strftime('%R') + ' - ' + hatch_times[5].strftime('%R'),
    ]
    for i in range(5):
        label =hatch_labels[i]

        legend_items.append(mpatches.Patch(facecolor='white', edgecolor='black',hatch=patterns[i], linewidth=1, label=label))



    ax1.legend(handles = legend_items)
    
    fig.tight_layout()
    fig.suptitle(pretty_name, fontsize=20)


    plt.show()
